In [6]:
import pandas as pd
URM_path = "data_train.csv"
URM_all_dataframe = pd.read_csv(filepath_or_buffer=URM_path,
                                sep=",",
                                # header=None,
                                dtype={0: int, 1: int, 2: int},
                                engine='python')

URM_all_dataframe.columns = ["UserID", "ItemID", "Interaction"]
import scipy.sparse as sps

mapped_id, original_id = pd.factorize(URM_all_dataframe["UserID"].unique())
user_original_ID_to_index = pd.Series(mapped_id, index=original_id)

mapped_id, original_id = pd.factorize(URM_all_dataframe["ItemID"].unique())
item_original_ID_to_index = pd.Series(mapped_id, index=original_id)

In [7]:
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Hybrid import *
from Recommenders.Neural.MultVAERecommender import MultVAERecommender_OptimizerMask
from Data_manager.UserUtils import *
import pandas as pd
import numpy as np

cutoff = 2

# Load the CSV file
file_path = 'submission_hybrids_14231.csv'
df = pd.read_csv(file_path)

URM_all = getURM_all()
users_few_iterations = URM_all.sum(axis=1)  
users_few_iterations = np.array(users_few_iterations).squeeze()
users_few_iterations = np.where((users_few_iterations < cutoff) * (users_few_iterations > 0))[0]

slim = SLIMElasticNetRecommender(URM_all)
slim.load_model(folder_path="slim_models/", file_name="slim_24")
rp3 = RP3betaRecommender(URM_all)
rp3.fit(alpha=0.20026352123406477, beta=0.15999879728761354, topK=32)
itemknn = ItemKNNCFRecommender(URM_all)
itemknn.fit(topK=31, shrink=435, similarity='tversky', normalize=True,
                                       feature_weighting='BM25', tversky_alpha=0.17113169506422393, tversky_beta=0.5684024974085575)
userknn = UserKNNCFRecommender(URM_all)
userknn.fit(topK=469, shrink=38, similarity='asymmetric', normalize=True,
                                       feature_weighting='TF-IDF', asymmetric_alpha=0.40077406933762383)
knn = TwoScoresHybridRecommender(URM_all, userknn, itemknn)
knn.fit(alpha=0.022195783788315104)

ease = EASE_R_Recommender(URM_all)
ease.load_model(folder_path="slim_models/", file_name="ease_all")

# top = LinearHybridRecommender(URM_all, [slim, rp3, knn, ease])
# top.fit(alphas=[0.8857667100747117, 0.6379807942443021, 0.0690929184825888, 0.11953478623354052])

mv = MultVAERecommender_OptimizerMask(URM_all)
mv.load_model(folder_path="slim_models/", file_name="MultVAE_all_opt_300")
epochs = 300
#mv.fit(epochs=epochs, learning_rate=0.0002229220517428562, l2_reg=1.7199457884691567e-05, dropout=0.5397666797136971, total_anneal_steps=134713, anneal_cap=0.20025495518554443,batch_size=512,encoding_size=460,next_layer_size_multiplier=10,max_n_hidden_layers=4, max_parameters=3000000000.0)

ials = IALSRecommender(URM_all)
ials.load_model(folder_path="slim_models/", file_name="IALS_all")

# top = ScoresHybridRecommender(URM_all, knn, mv, ials)
# top.fit(alpha=0.7509279131476281, beta=0.028318614437090585)

top = TwoScoresHybridRecommender(URM_all, slim, rp3)
top.fit(alpha=0.6201320790279279)


item_index_to_original_id = item_original_ID_to_index.reset_index().set_index(0).to_numpy().reshape(-1)

update_dict = {}
for user_id in users_few_iterations:
    recommendations = [item_index_to_original_id[index] for index in top.recommend(user_id, cutoff=10)]
    recommendations = " ".join(str(x) for x in recommendations)
    user_original = user_original_ID_to_index.reset_index().set_index(0).to_numpy().reshape(-1)[user_id]
    update_dict[user_original] = recommendations



SLIMElasticNetRecommender: Loading model from file 'slim_models/slim_24'
SLIMElasticNetRecommender: Loading complete
RP3betaRecommender: Similarity column 22222 (100.0%), 6494.17 column/sec. Elapsed time 3.42 sec
Similarity column 22222 (100.0%), 9334.68 column/sec. Elapsed time 2.38 sec
Similarity column 12638 (100.0%), 8688.59 column/sec. Elapsed time 1.45 sec
EASE_R_Recommender: Loading model from file 'slim_models/ease_all'
EASE_R_Recommender: Loading complete
MultVAERecommender: Loading model from file 'slim_models/MultVAE_all_opt_300'


2023-12-30 23:45:40.871758: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-12-30 23:45:40.872039: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


INFO:tensorflow:Restoring parameters from slim_models/MultVAE_all_opt_300/.session/session
MultVAERecommender: Loading complete
IALSRecommender: Loading model from file 'slim_models/IALS_all'
IALSRecommender: Loading complete


In [8]:
len(update_dict)

948

In [9]:
update_dict

{25: '7 51 15 18 4 107 85 2 10 88',
 31: '6 1 8 29 25 70 14 247 312 21',
 53: '859 3970 720 1424 3687 7075 6273 6472 4819 6471',
 66: '202 1 4 365 2 15 75 81 21 8',
 80: '10305 4855 16542 21 6 630 7110 7 1 274',
 105: '3 7 21 4 15 8752 9 6820 10 2',
 116: '15 101 7 1 3 2 6 4 34 8',
 120: '2 6 3 7 4 8 30 1 20 25',
 125: '15 101 7 1 3 2 6 4 34 8',
 161: '89 343 5 308 390 10 2 282 3 556',
 170: '450 224 361 184 558 288 319 35 382 11',
 186: '12 1 5 2 405 4 150 152 3 8',
 190: '2 3 4 1 6 8 112 15 11 10',
 195: '3565 3325 5744 3 7 36 4 262 10 1',
 209: '2 3 6 4 26 1 5 9 7 49',
 222: '4 1 67 2 172 8 6 20 29 238',
 228: '4 1 3 8 2 61 7 189 74 20',
 238: '4 2 1 80 8 6 20 174 22 33',
 296: '4 8 26 72 2 6 7 1 20 14',
 304: '2 4 211 45 7 22 3 48 33 95',
 313: '4168 8745 5728 6458 4819 5186 8168 7861 9701 15',
 335: '28 17 47 37 50 2 1 27 7 42',
 341: '7 380 422 949 2 3 359 6 30 56',
 353: '10 848 2 3 7 1 4 32 5 9',
 359: '153 285 213 14 59 26 22 8 2 1',
 378: '4 2 6 3 8 7 38 24 19 9',
 452: '1 4 

In [10]:
# Update the item_list for the specified userIDs
for user_id, new_item_list in update_dict.items():
    df.loc[df['user_id'] == user_id, 'item_list'] = new_item_list

# Save the updated dataframe back to a CSV file
df.to_csv('updated_submission.csv', index=False)